In [ ]:
import os
import paddle
import paddle.nn as nn
import paddle.vision as V
from paddle.io import DataLoader
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline
from tqdm import tqdm
from paddle import optimizer
from modules import UNet    # 模型
import logging
import numpy as np

logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", level=logging.INFO, datefmt="%I:%M:%S")

# 这里我们不需要用到图像标签，可以直接用paddle.vision里面提供的数据集接口
def get_data(args):
    transforms = V.transforms.Compose([
        V.transforms.Resize(80),  # args.image_size + 1/4 *args.image_size
        V.transforms.RandomResizedCrop(args.image_size, scale=(0.8, 1.0)),
        V.transforms.ToTensor(),
        V.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    dataset = V.datasets.ImageFolder(args.dataset_path, transform=transforms)
    dataloader = DataLoader(dataset, batch_size=args.batch_size, shuffle=True)
    return dataloader

class Diffusion:
    def __init__(self, noise_steps=500, beta_start=1e-4, beta_end=0.02, img_size=64, device="cuda"):
        self.noise_steps = noise_steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.img_size = img_size
        self.device = device

        self.beta = self.prepare_noise_schedule()
        self.alpha = 1. - self.beta
        self.alpha_hat = paddle.cumprod(self.alpha, dim=0)

    def prepare_noise_schedule(self):
        return paddle.linspace(self.beta_start, self.beta_end, self.noise_steps)

    def noise_images(self, x, t):
        sqrt_alpha_hat = paddle.sqrt(self.alpha_hat[t])[:, None, None, None]
        sqrt_one_minus_alpha_hat = paddle.sqrt(1 - self.alpha_hat[t])[:, None, None, None]
        Ɛ = paddle.randn(shape=x.shape)
        return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * Ɛ, Ɛ

    def sample_timesteps(self, n):
        return paddle.randint(low=1, high=self.noise_steps, shape=(n,))

    def sample(self, model, n):
        logging.info(f"Sampling {n} new images....")
        model.eval()
        with paddle.no_grad():
            x = paddle.randn((n, 3, self.img_size, self.img_size))
            for i in tqdm(reversed(range(1, self.noise_steps)), position=0):


                t = paddle.to_tensor([i] * x.shape[0]).astype("int64")
                # print(x.shape, t.shape)

                # print(f"完成第{i}步")
                predicted_noise = model(x, t)
                alpha = self.alpha[t][:, None, None, None]
                alpha_hat = self.alpha_hat[t][:, None, None, None]
                beta = self.beta[t][:, None, None, None]
                if i > 1:
                    noise = paddle.randn(shape=x.shape)
                else:
                    noise = paddle.zeros_like(x)
                x = 1 / paddle.sqrt(alpha) * (x - ((1 - alpha) / (paddle.sqrt(1 - alpha_hat))) * predicted_noise) + paddle.sqrt(beta) * noise
        model.train()
        x = (x.clip(-1, 1) + 1) / 2
        x = (x * 255)
        return x

In [ ]:
import paddle

if not os.path.exists('/dataset/OPIXray_Generate/'):
    os.mkdir('/dataset/OPIXray_Generate/')
if not os.path.exists('/dataset/OPIXray_Generate/train/'):
    os.mkdir('/dataset/OPIXray_Generate/train/')
if not os.path.exists('/dataset/OPIXray_Generate/test/'):
    os.mkdir('/dataset/OPIXray_Generate/test/')
if not os.path.exists('/dataset/OPIXray_Generate/val/'):
    os.mkdir('/dataset/OPIXray_Generate/val/')
if not os.path.exists('/dataset/OPIXray_Generate/meta/'):
    os.mkdir('/dataset/OPIXray_Generate/meta/')
if not os.path.exists('/dataset/OPIXray_Generate/crop/'):
    os.mkdir('/dataset/OPIXray_Generate/crop/')
if not os.path.exists('/dataset/OPIXray_Generate/images/'):
    os.mkdir('/dataset/OPIXray_Generate/images/')

model = UNet()
model.set_state_dict(paddle.load("models/Utility_Knife/ddpm_uncond980.pdparams"))   # 修改类型
diffusion = Diffusion(img_size=64, device="cuda")

# classes = {'Folding_Knife' : 0, 'Multi-tool_Knife' : 1, 'Scissor' : 2, 'Straight_Knife' : 3, 'Utility_Knife' : 4}
counts = 0
for count in range(625):
    sampled_images = diffusion.sample(model, n=32)
    files = open('/dataset/OPIXray_Generate/files.txt', 'a')
    # 采样图片
    for i in range(32):
        img = sampled_images[i].transpose([1, 2, 0])
        img = np.array(img).astype("uint8")
        new_img = Image.fromarray(np.uint8(img))
        pic_name = 'G04_{}.png'.format(counts + 1) # 修改类型对应的数字
        counts = counts + 1
        new_img.save('/dataset/OPIXray_Generate/images/' + pic_name)
        files.write(pic_name + ' ' + '4' + '\n') # 修改类型对应的数字
        # plt.subplot(2, 4,i+1)
        # plt.imshow(img)
    # plt.show()
files.close()